In [1]:
import snakeviz
import cProfile
import pstats

In [ ]:
import numpy as np
import copy
import time

In [ ]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [ ]:
X, Y, beta = generate_stdp_dataset(3, 10, 0, 1)

In [ ]:
X, Y= load_diabetes(return_X_y = True)

In [ ]:
print(Y[0:3])
Y = (Y - Y.mean()) / Y.std()
print(Y[0:3])
X = X / np.linalg.norm(X)
print(X[0:3])

In [ ]:
print(X)
print(Y)

In [ ]:
print(len(X[0]))

In [ ]:
X_train = copy.deepcopy(X)
Y_train = copy.deepcopy(Y[:len(Y) - 1])
X_train = [torch.tensor(member) for member in X_train]
Y_train = [torch.tensor(member) for member in Y_train]
dim = len(X[0])

In [ ]:
np.std(Y_train)

In [ ]:
global prev_mu_q

prev_mu_q = torch.zeros(dim, dtype=torch.float64)

In [ ]:
std0 = torch.eye(dim, dtype=torch.float64) * 0.3
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(10, dtype=torch.float64)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.MultivariateNormal(mu0, std0))
    # loop over the observed data
    for i in range(len(data)):
        pyro.sample("obs_{}".format(i), dist.Normal(f.dot(data[i][0]), 0.3), obs=data[i][1])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0.
    # - because we invoke constraints.positive, the optimizer
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    mu_q = pyro.param("mu_q", copy.deepcopy(prev_mu_q))
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.MultivariateNormal(mu_q, std0))

In [ ]:
def train_SVI(D_hat, n_steps):
    # setup the optimizer
    adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # do gradient steps
    for step in range(n_steps):
        svi.step(D_hat)
    
    breakpoint()
    mu_q = pyro.param("mu_q")
    return mu_q

In [ ]:
%load_ext snakeviz

In [ ]:
# %%snakeviz
y_hat = max(Y_train)
y_bottom = min(Y_train)
print(y_hat)
print(y_bottom)
conformal_set = []
step_size = 0.1
start = time.time()
while y_hat >= y_bottom:
    pyro.clear_param_store()
    # Create D_hat
    D_hat = list(zip(X_train[:-1], Y_train))
    D_hat.append((X_train[-1], y_hat))
    
    # Train SVI
    mu_q = train_SVI(D_hat, 10)
    prev_mu_q = mu_q
    
    # Calculate rank of y_hat
    rank = [(sum(D_hat[i][0] * mu_q) - D_hat[i][1]).detach().numpy() for i in range(len(D_hat))]
    y_hat_rank = rank[-1]
    
    # Add to conformal set if in not in bottom 10 percent of probabilities
    if np.count_nonzero(y_hat_rank > rank) / len(rank) > 0.1:
        conformal_set.append(copy.deepcopy(y_hat))
        print(f"{y_hat" Added})
    else:
        print(f"{y_hat} Not added")
        
    y_hat -= step_size
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

In [ ]:
print(conformal_set)
print(f"Y[-1]: {Y[-1]}")
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")